In [ ]:
import os
import glob
import shutil
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Move all CSV files to a backup folder
csv_files = glob.glob("/content/drive/MyDrive/Colab Notebooks/PlusW/Lecture 5/csv_files/*.csv")
for file in csv_files:
    shutil.move(file, "/content/drive/MyDrive/Colab Notebooks/PlusW/Lecture 5/backup_folder/")
    print(f"Moved file: {file}")

# Automating Export Function
def export_data(df, filename, format):
    if format == "csv":
        df.to_csv(filename, index=False)
        print(f"Data exported to {filename} in CSV format.")
    elif format == "json":
        df.to_json(filename, orient="records")
        print(f"Data exported to {filename} in JSON format.")
    else:
        print("Unsupported format.")

# Example usage:
# Creating a sample dataframe
data = {'Name': ['Alice', 'Bob', 'Charlie'],
        'Age': [25, 30, 35],
        'City': ['New York', 'Los Angeles', 'Chicago']}
df = pd.DataFrame(data)

# Exporting to CSV
export_data(df, "output.csv", "csv")

# Exporting to JSON
export_data(df, "output.json", "json")


Moved file: /content/drive/MyDrive/Colab Notebooks/PlusW/Lecture 5/csv_files/data.csv
Moved file: /content/drive/MyDrive/Colab Notebooks/PlusW/Lecture 5/csv_files/data1.csv
Moved file: /content/drive/MyDrive/Colab Notebooks/PlusW/Lecture 5/csv_files/data2.csv
Data exported to output.csv in CSV format.
Data exported to output.json in JSON format.


In [12]:
import yfinance as yf
import sqlite3
import pandas as pd
import time

In [14]:
# Database setup
db_name = "/content/drive/MyDrive/Colab Notebooks/PlusW/Lecture 5/stocks.db"
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS stock_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    symbol TEXT,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
    open REAL,
    high REAL,
    low REAL,
    close REAL,
    volume INTEGER)''')
conn.commit()

# Function to fetch stock data
def fetch_stock_data(symbol):
    try:
        stock = yf.Ticker(symbol)
        data = stock.history(period="1d", interval="1m")

        if data.empty:
            print(f"No data found for {symbol}. Skipping...")
            return None  # Return None if no data is available

        latest = data.iloc[-1]  # Get the most recent price data
        return {
            "symbol": symbol,
            "open": latest["Open"],
            "high": latest["High"],
            "low": latest["Low"],
            "close": latest["Close"],
            "volume": latest["Volume"]
        }
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

# Function to store data in SQLite
def store_data(symbol):
    stock_data = fetch_stock_data(symbol)

    if stock_data:  # Only store if data is available
        cursor.execute('''INSERT INTO stock_data (symbol, open, high, low, close, volume)
                        VALUES (?, ?, ?, ?, ?, ?)''',
                       (stock_data["symbol"], stock_data["open"],
                        stock_data["high"], stock_data["low"],
                        stock_data["close"], stock_data["volume"]))
        conn.commit()  # Commit after insert
        print(f"Stored data for {symbol}")

# Function to analyze stock data
def analyze_stock(symbol):
    df = pd.read_sql_query("SELECT * FROM stock_data WHERE symbol=? ORDER BY timestamp DESC LIMIT 100", conn, params=(symbol,))
    print(df)

# Example Usage
symbol = "Gold"  # Gold asset
for _ in range(5):  # Fetch data 5 times with intervals
    store_data(symbol)
    time.sleep(60)  # Wait for 1 minute before fetching again

analyze_stock(symbol)

# Close database connection
conn.close()

Stored data for Gold
Stored data for Gold
Stored data for Gold
Stored data for Gold
Stored data for Gold
   id symbol            timestamp       open       high    low  close  volume
0   5   Gold  2025-02-25 11:20:56  18.264999  18.280001  18.26  18.27  473488
1   4   Gold  2025-02-25 11:19:56  18.264999  18.280001  18.26  18.27  473488
2   3   Gold  2025-02-25 11:18:56  18.264999  18.280001  18.26  18.27  473488
3   2   Gold  2025-02-25 11:17:56  18.264999  18.280001  18.26  18.27  473488
4   1   Gold  2025-02-25 11:16:56  18.264999  18.280001  18.26  18.27  473488


In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [22]:
URL = "https://boards.4chan.org/g/"
HEADERS = {"User-Agent": "Mozilla/5.0"}

def get_posts(url):
    response = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(response.text, "html.parser")
    threads = soup.find_all("div", class_="thread")
    post_list = []
    for thread in threads:
        title = thread.find("span", class_="subject").text if thread.find("span", class_="subject") else "No title"
        poster = thread.find("span", class_="name").text if thread.find("span", class_="name") else "Anonymous"
        post_content = thread.find("blockquote").text.strip() if thread.find("blockquote") else "No content"
        post_list.append({"Title": title, "Poster": poster, "Content": post_content})
    return post_list

posts_data = get_posts(URL)
df = pd.DataFrame(posts_data)
df.to_csv("/content/drive/MyDrive/Colab Notebooks/PlusW/Lecture 5/4chan_posts.csv", index=False)
print("Data saved to 4chan_posts.csv")

Data saved to 4chan_posts.csv
